In [20]:
import requests

url = "http://127.0.0.1:8777/api/tools/call"
headers = {"Content-Type": "application/json", "X-AdaOS-Token": "dev-local-token"}  # подставь актуальный токен
payload = {"tool": "weather_skill:get_weather", "arguments": {"city": "Berlin"}}

response = requests.post(url, json=payload, headers=headers)

print("Status:", response.status_code)
try:
    print("Response:", response.json())
except ValueError:
    print("Raw Response:", response.text)

Status: 200
Response: {'ok': True, 'result': {'ok': True, 'city': 'Berlin', 'temp': 18.13, 'description': 'few clouds'}, 'trace_id': '90842474-73e9-43b8-b11a-478babcbcfaa'}


In [12]:
!curl -i http://127.0.0.1:8777/health/live

HTTP/1.1 200 OK
date: Sun, 21 Sep 2025 18:51:26 GMT
server: uvicorn
content-length: 11
content-type: application/json

{"ok":true}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    11  100    11    0     0   1346      0 --:--:-- --:--:-- --:--:--  1375


In [13]:
!curl -H "X-AdaOS-Token: dev-local-token" http://127.0.0.1:8777/api/subnet/nodes


{"ok":true,"nodes":[]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    22  100    22    0     0   2794      0 --:--:-- --:--:-- --:--:--  3142


In [14]:
print(requests.get("http://127.0.0.1:8777/api/node/status", headers=headers).json())

{'node_id': 'ddf3d05d-4903-4c97-b634-b3346795fecc', 'subnet_id': '2a1db355-25df-49c6-a2e1-e6126ed828ea', 'role': 'hub', 'hub_url': None, 'ready': True}


In [16]:
import requests

headers = {"X-AdaOS-Token": "dev-local-token"}

# 1) проверить статус
print(requests.get("http://127.0.0.1:8778/api/node/status", headers=headers).json())

# 2) сменить роль на member или hub (и задать hub_url)
payload = {"role": "hub", "hub_url": "http://127.0.0.1:8777"}
print(requests.post("http://127.0.0.1:8778/api/node/role", json=payload, headers=headers).json())

# 3) снова статус — должен быть role=member, ready=true
print(requests.get("http://127.0.0.1:8778/api/node/status", headers=headers).json())

{'node_id': '085202cf-41e2-41be-bb19-bc0ed0f1c34e', 'subnet_id': '2a1db355-25df-49c6-a2e1-e6126ed828ea', 'role': 'hub', 'hub_url': None, 'ready': True}
{'ok': True, 'node': {'node_id': '085202cf-41e2-41be-bb19-bc0ed0f1c34e', 'subnet_id': '2a1db355-25df-49c6-a2e1-e6126ed828ea', 'role': 'hub', 'hub_url': None, 'ready': True}, 'diagnostics': {'requested_role': 'hub', 'hub_url': 'http://127.0.0.1:8777', 'subnet_id_used': None, 'now_ready': True}}
{'node_id': '085202cf-41e2-41be-bb19-bc0ed0f1c34e', 'subnet_id': '2a1db355-25df-49c6-a2e1-e6126ed828ea', 'role': 'hub', 'hub_url': None, 'ready': True}


In [17]:
import requests

headers = {"X-AdaOS-Token": "dev-local-token"}
hub_status = requests.get("http://127.0.0.1:8777/api/node/status", headers=headers).json()
print(hub_status["subnet_id"])

2a1db355-25df-49c6-a2e1-e6126ed828ea


In [18]:
payload = {"role": "hub", "hub_url": "http://127.0.0.1:8777", "subnet_id": hub_status["subnet_id"]}
print(requests.post("http://127.0.0.1:8778/api/node/role", json=payload, headers=headers).json())

{'ok': True, 'node': {'node_id': '085202cf-41e2-41be-bb19-bc0ed0f1c34e', 'subnet_id': '2a1db355-25df-49c6-a2e1-e6126ed828ea', 'role': 'hub', 'hub_url': None, 'ready': True}, 'diagnostics': {'requested_role': 'hub', 'hub_url': 'http://127.0.0.1:8777', 'subnet_id_used': '2a1db355-25df-49c6-a2e1-e6126ed828ea', 'now_ready': True}}


In [19]:
!curl -H "X-AdaOS-Token: dev-local-token" http://127.0.0.1:8777/api/subnet/nodes


{"ok":true,"nodes":[]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    22  100    22    0     0   2825      0 --:--:-- --:--:-- --:--:--  3142


In [11]:
# AdaOS Observability v0 — sanity check notebook
#
# How to use:
# 1) Set HUB_URL, MEMBER_URL, TOKEN below for your setup.
# 2) Run this single cell. It will:
#    - ping health/live & health/ready
#    - show node status for hub & member
#    - list subnet nodes from hub
#    - (optionally) call a tool to generate events (TOOL_CALL_ENABLED)
#    - tail last N lines from hub /api/observe/tail
#
# All steps are guarded; failures are shown but won't stop the run.

import os, json, time, textwrap, pathlib, sys
from typing import Optional, Dict, Any, List

import requests

# ----------------- CONFIG -----------------
HUB_URL = os.environ.get("ADAOS_HUB_URL", "http://127.0.0.1:8777")
MEMBER_URL = os.environ.get("ADAOS_MEMBER_URL", "http://127.0.0.1:8778")
TOKEN = os.environ.get("ADAOS_TOKEN", "dev-local-token")

# If you have a working tool (e.g., weather_skill:get_weather) set this True
TOOL_CALL_ENABLED = True
TOOL_SPEC = "weather_skill:get_weather"
TOOL_ARGS = {"city": "Berlin"}

TAIL_LINES = 120
TOPIC_FILTER = None  # e.g. "ui." or "net.subnet." or None

# ------------------------------------------

HEADERS = {"X-AdaOS-Token": TOKEN, "Content-Type": "application/json"}


def hr(title: str):
    print("\n" + "=" * 20 + f" {title} " + "=" * 20)


def req_json(method: str, base: str, path: str, **kwargs) -> Dict[str, Any]:
    url = base.rstrip("/") + path
    try:
        r = requests.request(method, url, headers=HEADERS, timeout=5, **kwargs)
        try:
            data = r.json()
        except Exception:
            data = {"_raw": r.text}
        return {"ok": r.ok, "status": r.status_code, "url": url, "data": data, "headers": dict(r.headers)}
    except Exception as e:
        return {"ok": False, "status": None, "url": url, "error": str(e)}


def pretty_print(label: str, payload: Any):
    print(f"\n{label}:")
    if isinstance(payload, (dict, list)):
        print(json.dumps(payload, ensure_ascii=False, indent=2))
    else:
        print(payload)


# 1) HEALTH
hr("HEALTH (hub & member)")
hub_live = req_json("GET", HUB_URL, "/health/live")
hub_ready = req_json("GET", HUB_URL, "/health/ready")
mem_live = req_json("GET", MEMBER_URL, "/health/live")
mem_ready = req_json("GET", MEMBER_URL, "/health/ready")

pretty_print("hub/live", hub_live)
pretty_print("hub/ready", hub_ready)
pretty_print("member/live", mem_live)
pretty_print("member/ready", mem_ready)

# 2) NODE STATUS
hr("NODE STATUS (hub & member)")
hub_status = req_json("GET", HUB_URL, "/api/node/status")
mem_status = req_json("GET", MEMBER_URL, "/api/node/status")
pretty_print("hub status", hub_status)
pretty_print("member status", mem_status)

# 3) SUBNET NODES (from hub)
hr("SUBNET /api/subnet/nodes")
nodes = req_json("GET", HUB_URL, "/api/subnet/nodes")
pretty_print("nodes", nodes)

# 4) (Optional) TOOL CALL to generate events & trace
if TOOL_CALL_ENABLED:
    hr(f"TOOLS CALL → {TOOL_SPEC}")
    tool_resp = req_json("POST", MEMBER_URL, "/api/tools/call", json={"tool": TOOL_SPEC, "arguments": TOOL_ARGS})
    pretty_print("tool call response", tool_resp)
else:
    hr("TOOLS CALL skipped (TOOL_CALL_ENABLED=False)")

# 5) OBSERVE TAIL from hub
hr(f"OBSERVE TAIL last {TAIL_LINES} lines (hub)")
tail = req_json("GET", HUB_URL, f"/api/observe/tail?lines={TAIL_LINES}")
lines = []
if tail.get("ok") and isinstance(tail.get("data"), dict):
    lines = tail["data"].get("lines") or []
    # Optional filter by topic prefix
    if TOPIC_FILTER:
        keep = []
        for ln in lines:
            try:
                obj = json.loads(ln)
                if str(obj.get("topic", "")).startswith(TOPIC_FILTER):
                    keep.append(ln)
            except Exception:
                # keep raw if parsing fails and no filter
                pass
        lines = keep

print(f"tail lines: {len(lines)}")
for ln in lines:
    print(ln)

# 6) SUMMARY
hr("SUMMARY")
summary = {
    "hub_ready": hub_ready.get("ok") and hub_ready.get("status") == 200,
    "member_ready": mem_ready.get("ok") and mem_ready.get("status") == 200,
    "nodes_seen_on_hub": len(nodes.get("data", {}).get("nodes", [])) if isinstance(nodes.get("data"), dict) else None,
    "tool_call_status": hub_ready.get("status") if not TOOL_CALL_ENABLED else (tool_resp.get("status")),
    "tail_lines": len(lines),
}
pretty_print("summary", summary)


==================== HEALTH (hub & member) ====================

hub/live:
{
  "ok": true,
  "status": 200,
  "url": "http://127.0.0.1:8777/health/live",
  "data": {
    "ok": true
  },
  "headers": {
    "Connection": "close",
    "Content-Length": "11",
    "Content-Type": "application/json",
    "Date": "Sun, 21 Sep 2025 18:50:30 GMT",
    "Server": "uvicorn"
  }
}

hub/ready:
{
  "ok": true,
  "status": 200,
  "url": "http://127.0.0.1:8777/health/ready",
  "data": {
    "ok": true
  },
  "headers": {
    "Connection": "close",
    "Content-Length": "11",
    "Content-Type": "application/json",
    "Date": "Sun, 21 Sep 2025 18:50:30 GMT",
    "Server": "uvicorn"
  }
}

member/live:
{
  "ok": true,
  "status": 200,
  "url": "http://127.0.0.1:8778/health/live",
  "data": {
    "ok": true
  },
  "headers": {
    "Connection": "close",
    "Content-Length": "11",
    "Content-Type": "application/json",
    "Date": "Sun, 21 Sep 2025 18:50:29 GMT",
    "Server": "uvicorn"
  }
}

member/r

In [29]:
import os, json, time, requests

HUB_URL = os.environ.get("ADAOS_HUB_URL", "http://127.0.0.1:8777")
MEMBER_URL = os.environ.get("ADAOS_MEMBER_URL", "http://127.0.0.1:8778")
TOKEN = os.environ.get("ADAOS_TOKEN", "dev-local-token")

HEADERS = {"X-AdaOS-Token": TOKEN, "Content-Type": "application/json"}


def call(method, base, path, **kw):
    url = base.rstrip("/") + path
    try:
        r = requests.request(method, url, headers=HEADERS, timeout=5, **kw)
        try:
            data = r.json()
        except Exception:
            data = {"_raw": r.text}
        return r.status_code, data
    except Exception as e:
        return None, {"error": str(e)}


print("health hub:", call("GET", HUB_URL, "/health/ready"))
print("health member:", call("GET", MEMBER_URL, "/health/ready"))
print("hub status:", call("GET", HUB_URL, "/api/node/status"))
print("member status:", call("GET", MEMBER_URL, "/api/node/status"))
print("hub nodes:", call("GET", HUB_URL, "/api/subnet/nodes"))

# генерируем тестовое событие на member → оно попадёт в локальный лог и уйдёт батчем на hub
print("emit test (member):", call("POST", MEMBER_URL, "/api/observe/test", json={"kind": "ping", "note": "from-notebook"}))

# даём время батчеру переслать
time.sleep(2.0)

# хвост логов с hub
status, tail = call("GET", HUB_URL, "/api/observe/tail?lines=1000")
lines = tail.get("lines") or []
print("tail lines:", len(lines))

# фильтруем по нашему тестовому топику
obs = []
for ln in lines:
    try:
        evt = json.loads(ln)
        if evt.get("topic") == "obs.test.ping":
            obs.append(evt)
    except Exception:
        pass

print("found obs.test.ping:", len(obs))
if obs:
    print(json.dumps(obs[-1], ensure_ascii=False, indent=2))

health hub: (200, {'ok': True})
health member: (200, {'ok': True})
hub status: (200, {'node_id': '2f29994a-131a-4a29-9d51-bb3b235e8574', 'subnet_id': '180de7f4-b7ee-4116-b3c5-a5b9a8e2e22f', 'role': 'hub', 'hub_url': None, 'ready': True})
member status: (200, {'node_id': '2f29994a-131a-4a29-9d51-bb3b235e8574', 'subnet_id': '180de7f4-b7ee-4116-b3c5-a5b9a8e2e22f', 'role': 'hub', 'hub_url': None, 'ready': True})
hub nodes: (200, {'ok': True, 'nodes': []})
emit test (member): (200, {'ok': True, 'topic': 'obs.test.ping', 'payload': {'note': 'hello', 'at': 1758022209.0505576}})
tail lines: 62
found obs.test.ping: 1
{
  "ts": 1758022209.0544453,
  "topic": "obs.test.ping",
  "payload": {
    "note": "hello",
    "at": 1758022209.0505576
  },
  "trace": "7d116eae-117d-4cb0-8c72-d0530c07d5c4",
  "source": "observe_api",
  "actor": "diagnostics",
  "node_id": "2f29994a-131a-4a29-9d51-bb3b235e8574",
  "role": "hub"
}


In [30]:
!curl -H "X-AdaOS-Token: dev-local-token" -X POST http://127.0.0.1:8777/api/observe/test

{"ok":true,"topic":"obs.test.ping","payload":{"note":"hello","at":1758022218.5667012}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0  15280      0 --:--:-- --:--:-- --:--:-- 17200


In [31]:
from pathlib import Path

IMPORTANT = {"src", "adaos", "tests", "docs"}


def list_clean_tree(root="."):
    root_path = Path(root).resolve()

    def walk(path: Path, depth=0, max_depth=4):
        if depth > max_depth:
            return
        for p in sorted(path.iterdir()):
            if p.is_dir() and not p.name.startswith((".", "__")) and "egg-info" not in p.name:
                rel = p.relative_to(root_path)
                if any(part in IMPORTANT for part in rel.parts) or depth == 0:
                    print("  " * depth + f"- {rel}")
                walk(p, depth + 1, max_depth)

    print(f"Clean project tree at {root_path}:")
    walk(root_path)


if __name__ == "__main__":
    list_clean_tree("c:/git/MUIV/adaos/src/adaos/sdk")

Clean project tree at C:\git\MUIV\adaos\src\adaos\sdk:
- descriptions
- llm
- scenarios
- skills
- utils
- validation


In [32]:
from pathlib import Path

IMPORTANT = {"src", "adaos", "tests", "docs"}


def list_clean_tree(root=".", include_files: bool = False, max_depth: int = 4):
    """
    Печатает дерево проекта. По умолчанию выводит только папки.
    Если include_files=True, дополнительно выводит файлы.
    Правило отбора такое же, как было: на глубине 0 показываем всё;
    глубже — только если в относительном пути встретится один из IMPORTANT.
    """
    root_path = Path(root).resolve()

    def should_print(rel_parts: tuple[str, ...], is_dir: bool, depth: int) -> bool:
        if depth == 0:
            return True
        parts_to_check = rel_parts if is_dir else rel_parts[:-1]  # для файла проверяем только родительские папки
        return any(part in IMPORTANT for part in parts_to_check)

    def walk(path: Path, depth: int = 0):
        if depth > max_depth:
            return
        for p in sorted(path.iterdir()):
            name = p.name
            if name.startswith((".", "__")) or "egg-info" in name:
                continue

            rel = p.relative_to(root_path)

            if p.is_dir():
                if should_print(rel.parts, is_dir=True, depth=depth):
                    print("  " * depth + f"- {rel}")
                walk(p, depth + 1)
            elif include_files and p.is_file():
                if should_print(rel.parts, is_dir=False, depth=depth):
                    print("  " * depth + f"  {rel}")

    print(f"Clean project tree at {root_path}:")
    walk(root_path)


if __name__ == "__main__":
    # пример: показать и каталоги, и файлы
    list_clean_tree("c:/git/MUIV/adaos/src/adaos/sdk", include_files=True)

Clean project tree at C:\git\MUIV\adaos\src\adaos\sdk:
  a_permissions.py
  bus.py
  context.py
  decorators.py
- descriptions
  env.py
  errors.py
  exporter.py
  i18n.py
- llm
  scenario_service.py
- scenarios
  skill_api.py
  skill_memory.py
- skills
  types.py
- utils
- validation


# Scenarios

In [21]:
import requests

H = {"X-AdaOS-Token": "dev-local-token"}

In [22]:
r = requests.post(
    "http://127.0.0.1:8777/api/scenarios/create",
    json={"id": "introduction1", "template": "template"},
    headers=H,
)
print(r.status_code, r.text)

405 {"detail":"Method Not Allowed"}


In [31]:
# установить сценарий из репо
requests.post("http://127.0.0.1:8777/api/scenarios/install/", json={"ref": "main", "name": "greet_on_boot"}, headers=H).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
requests.delete("http://127.0.0.1:8777/api/scenarios/morning", json={"ref": "main", "name": "morning"}, headers=H).json()

{'ok': True}

In [ ]:
requests.get("http://127.0.0.1:8777/api/scenarios/list", headers=H).json()

{'items': []}